# Importing Libraries

In [1]:
# Importing Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Get Data from the fulldataset source

This data set has undergone the preliminary cleaning from the python packages, dataprep.py

In [29]:
# Importing Files
fulldataset_df = pd.read_csv('../road_to_qatar_2022/data/fulldataset.csv')
data_cups = pd.read_csv('../raw_data/data/FIFA World Cup/WorldCups.csv')
teams_ranking_df = pd.read_csv('../road_to_qatar_2022/data/teamsranking.csv')

# Slearn Libraries

In [3]:
# Analysis of Worldcup Matches
# Slearn Libraries
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV

# Count champions 

In [4]:
# Count champions 
winners =  data_cups['Winner'].map(lambda n: 'Germany' if n == 'Germany FR' else n).value_counts()


# Home and Away Team champions
#data_matches_to_use["Home Team Champion"] = 0
#data_matches_to_use["Away Team Champion"] = 0
fulldataset_df["Home Team Champion"] = 0
fulldataset_df["Away Team Champion"] = 0

def winners_wc(df):
    if(winners.get(df['Home Team Name']) != None):
        df['Home Team Champion'] = winners.get(df['Home Team Name'])
    if(winners.get(df['Away Team Name']) != None):
        df['Away Team Champion'] = winners.get(df['Away Team Name'])
    
    return df

#data_matches_to_use = data_matches_to_use.apply(winners_wc, axis = 1)
fulldataset_df = fulldataset_df.apply(winners_wc, axis = 1)


In [5]:
winners.head()

Brazil       5
Italy        4
Germany      4
Uruguay      2
Argentina    2
Name: Winner, dtype: int64

In [8]:
fulldataset_df.head()

,Date,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Winner,home_team_rank,home_team_points,home_team_previous_points,away_team_rank,away_team_points,away_team_previous_points,Home Team Champion,Away Team Champion
0,1930-07-13,France,4.0,1.0,Mexico,1,4,1759.78,1764.85,13.0,1644.89,1649.57,1,0
1,1930-07-13,USA,3.0,0.0,Belgium,1,16,1627.48,1635.01,2.0,1816.71,1821.92,0,0
2,1930-07-14,Yugoslavia,2.0,1.0,Brazil,1,6,64.00,64.00,1.0,1841.30,1837.56,0,5
3,1930-07-14,Romania,3.0,1.0,Peru,1,53,1434.68,1427.84,23.0,1561.01,1562.24,0,0
4,1930-07-15,Argentina,1.0,0.0,France,1,3,1773.88,1770.65,4.0,1759.78,1764.85,2,1


In [9]:
fulldataset_df[fulldataset_df['Away Team Name']=='Qatar']

,Date,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Winner,home_team_rank,home_team_points,home_team_previous_points,away_team_rank,away_team_points,away_team_previous_points,Home Team Champion,Away Team Champion
894,1984-03-28,Iraq,1.0,1.0,Qatar,0,68,1343.01,1338.91,50.0,1439.89,1441.97,0,0
902,1985-07-10,Bahrain,1.0,1.0,Qatar,0,85,1280.64,1289.25,50.0,1439.89,1441.97,0,0
903,1985-07-12,Saudi Arabia,0.0,0.0,Qatar,0,51,1437.78,1435.74,50.0,1439.89,1441.97,0,0


# Replace Team names by corresponding id:

## Re creating the list of teams since the dataset has changed as a result of the mrege with international matches

In [10]:
# Creating a list of all teams playing football worlcup:
team_name = {}
index = 0
for idx, row in fulldataset_df.iterrows():
    name = row['Home Team Name']
    if (name not in team_name.keys()):
        team_name[name] = index
        index += 1
        
    name = row['Away Team Name']
    if (name not in team_name.keys()):
        team_name[name] = index
        index += 1

In [11]:
# Replace Team names by corresponding id:
def replace_name(df):
    df['Home Team Name'] = team_name[df['Home Team Name']]
    df['Away Team Name'] = team_name[df['Away Team Name']]
    
    return df

#Team_id = data_matches_to_use.apply(replace_name, axis = 1)
Team_id = fulldataset_df.apply(replace_name, axis = 1)

##  Now we dont need number of goals and year

In [12]:
Team_id = Team_id.drop(['Date','Home Team Goals', 'Away Team Goals'], 1)

/tmp/ipykernel_30404/953902738.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  Team_id = Team_id.drop(['Date','Home Team Goals', 'Away Team Goals'], 1)


In [13]:
Team_id= Team_id[['Home Team Name',
                 'Away Team Name',
                 'home_team_rank',
                 'home_team_points',
                 'home_team_previous_points',
                 'away_team_rank',
                 'away_team_points',
                 'away_team_previous_points',
                 'Home Team Champion',
                 'Away Team Champion',
                 'Winner']]

In [14]:
Team_id=Team_id.fillna(0)
Team_id.isnull().sum().sum()

0

# Preparing Training and Testing Data:

In [15]:
# Preparing Training and Testing Data:
X = Team_id.iloc[:, 0:10].values
y = Team_id.iloc[:, -1].values

X = np.array(X, dtype = 'f')
y = np.array(y, dtype = 'f')

from sklearn.preprocessing import LabelEncoder
LabelEncoder_res=LabelEncoder()
y=LabelEncoder_res.fit_transform(y)

X, y = shuffle(X,y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

# Scalling of Data:

In [16]:
# Scalling of Data:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

# Building ANN

In [20]:
# Building ANN
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import classification_report, confusion_matrix

# seed

In [21]:
#seed
np.random.seed(47)

# Random Forest

In [22]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
rc = RandomForestClassifier(n_estimators=100)
rc.fit(X_train, y_train)
score_train_acc = rc.score(X_train, y_train)
score_test_acc = rc.score(X_test, y_test)
print(score_train_acc) #91.58
print(score_test_acc)  #51.31
y_pred_rc = rc.predict(X_test) 
print(classification_report(y_test, y_pred_rc))
print(confusion_matrix(y_test, y_pred_rc, labels=range(3)))

0.9225181598062954
0.5819209039548022
              precision    recall  f1-score   support

           0       0.65      0.58      0.61       164
           1       0.56      0.70      0.62       145
           2       0.35      0.20      0.25        45

    accuracy                           0.58       354
   macro avg       0.52      0.49      0.50       354
weighted avg       0.58      0.58      0.57       354

[[ 95  61   8]
 [ 34 102   9]
 [ 17  19   9]]


# XGBoost

In [23]:
import xgboost as xgb

In [24]:
X_test

array([[-0.30391842, -0.48135483,  1.0616212 , ...,  0.51092595,
        -0.62287086,  2.6774995 ],
       [ 0.0379898 ,  0.0875288 ,  0.10519473, ...,  0.34498698,
        -0.62287086, -0.45843044],
       [ 0.2659286 , -1.012313  , -0.6352645 , ...,  1.0508553 ,
        -0.62287086,  3.4614818 ],
       ...,
       [ 0.2659286 , -0.86061066, -0.6352645 , ...,  0.07529466,
        -0.62287086, -0.45843044],
       [ 0.15195921,  1.0356683 , -0.3884448 , ..., -0.14663258,
        -0.62287086, -0.45843044],
       [-0.1139694 ,  0.20130554, -0.75867444, ...,  0.5704514 ,
        -0.04414045, -0.45843044]], dtype=float32)

In [25]:
# XGBoost

XGB = xgb.XGBClassifier(base_score=0.5, 
                        booster='gbtree', 
                        colsample_bylevel=1,
                        colsample_bytree=0.8, 
                        gamma=0.4, 
                        learning_rate=0.01,
                        max_delta_step=0, 
                        max_depth=3, 
                        min_child_weight=1, 
                        #missing=null,
                        n_estimators=40, 
                        n_jobs=1, 
                        nthread=None, 
                        objective='multi:softprob',
                        random_state=0, 
                        reg_alpha=1e-05, 
                        reg_lambda=1, 
                        scale_pos_weight=1,
                        seed=2, 
                        silent=True, 
                        subsample=0.8)
#XGB = xgb.XGBClassifier(user_label_encoder=False)
XGB.fit(X_train, y_train)
score_train_acc = XGB.score(X_train, y_train)
score_test_acc = XGB.score(X_test, y_test)
print(score_train_acc) #66.53
print(score_test_acc)  #56.59      
y_pred_XGB = XGB.predict(X_test)
print(classification_report(y_test, y_pred_XGB))
print(confusion_matrix(y_test, y_pred_XGB, labels=range(3)))

[22:00:00] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

0.612590799031477
0.6271186440677966
              precision    recall  f1-score   support

           0       0.81      0.48      0.60       164
           1       0.55      0.91      0.68       145
           2       0.71      0.27      0.39        45

    accuracy                           0.63       354
   macro avg       0.69      0.55      0.56       354
weighted avg       0.69      0.63      0.61       354

[[ 78  83   3]
 [ 11 132   2]
 [  7  26  12]]


# Create the parameters list you wish to tune

In [26]:
parameters = { 'learning_rate' : [0.001,0.01, 0.1, 1],
               'n_estimators' : [40, 100],
               'max_depth': [3, 6],
               'min_child_weight': [1, 3],
               'gamma':[0.4],
               'subsample' : [0.5, 0.8],
               'colsample_bytree' : [0.8],
               'scale_pos_weight' : [1],
               'reg_alpha':[1e-5]
             } 

clf = xgb.XGBClassifier()

grid_obj = GridSearchCV(clf,
                        param_grid=parameters,
                        cv=5)
grid_obj = grid_obj.fit(X_train,y_train)
clf = grid_obj.best_estimator_
print(clf)


[22:00:24] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[22:00:24] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[22:00:24] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[22:00:24] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[22:00:24] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[22:00:24] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[22:00:25] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[22:00:25] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[22:00:25] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[22:00:25] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[22:00:25] WARNING: ../src/learner.cc:767: 
Parameters: { "s

[22:00:41] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[22:00:41] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[22:00:41] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[22:00:41] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[22:00:41] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[22:00:41] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[22:00:42] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[22:00:42] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[22:00:42] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[22:00:42] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[22:00:42] WARNING: ../src/learner.cc:767: 
Parameters: { "s

[22:00:55] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[22:00:55] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[22:00:55] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[22:00:55] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[22:00:55] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[22:00:55] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[22:00:55] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[22:00:56] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[22:00:56] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[22:00:56] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[22:00:56] WARNING: ../src/learner.cc:767: 
Parameters: { "s

[22:01:08] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[22:01:08] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[22:01:08] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[22:01:08] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[22:01:08] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[22:01:09] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[22:01:09] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[22:01:09] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[22:01:09] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[22:01:09] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight" } are not used.

[22:01:09] WARNING: ../src/learner.cc:767: 
Parameters: { "s

In [27]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}


clf_1 = RandomForestClassifier()

grid_obj1 = GridSearchCV(estimator = clf_1, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
grid_obj1 = grid_obj1.fit(X_train,y_train)
clf_1 = grid_obj1.best_estimator_
print(clf_1)

Fitting 3 folds for each of 288 candidates, totalling 864 fits
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100; total time=   1.0s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=200; total time=   1.1s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=300; total time=   1.6s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=10, n_estimators=100; total time=   0.5s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=10, n_estimators=100; total time=   0.5s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=10, n_estimators=100; total time=   0.6s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=10, n_estimators=200; total time=   2.0s
[C

[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100; total time=   1.1s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=200; total time=   1.1s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=1000; total time=   7.3s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=10, n_estimators=300; total time=   2.1s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=10, n_estimators=1000; total time=   4.2s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=12, n_estimators=300; total time=   1.3s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=12, n_estimators=1000; total time=   4.2s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_sam

[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100; total time=   1.1s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=300; total time=   1.6s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=1000; total time=   7.8s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=10, n_estimators=300; total time=   1.6s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=10, n_estimators=1000; total time=   4.1s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=12, n_estimators=1000; total time=   4.1s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=4, min_samples_split=8, n_estimators=300; total time=   1.3s
[CV] END bootstrap=True, max_depth=80, max_features=2, min_samp

[CV] END bootstrap=True, max_depth=90, max_features=2, min_samples_leaf=5, min_samples_split=12, n_estimators=300; total time=   1.2s
[CV] END bootstrap=True, max_depth=90, max_features=2, min_samples_leaf=5, min_samples_split=12, n_estimators=1000; total time=   4.2s
[CV] END bootstrap=True, max_depth=90, max_features=3, min_samples_leaf=3, min_samples_split=8, n_estimators=300; total time=   1.3s
[CV] END bootstrap=True, max_depth=90, max_features=3, min_samples_leaf=3, min_samples_split=10, n_estimators=100; total time=   0.5s
[CV] END bootstrap=True, max_depth=90, max_features=3, min_samples_leaf=3, min_samples_split=10, n_estimators=100; total time=   0.4s
[CV] END bootstrap=True, max_depth=90, max_features=3, min_samples_leaf=3, min_samples_split=10, n_estimators=100; total time=   0.4s
[CV] END bootstrap=True, max_depth=90, max_features=3, min_samples_leaf=3, min_samples_split=10, n_estimators=200; total time=   0.9s
[CV] END bootstrap=True, max_depth=90, max_features=3, min_sam

[CV] END bootstrap=True, max_depth=90, max_features=3, min_samples_leaf=3, min_samples_split=8, n_estimators=300; total time=   1.3s
[CV] END bootstrap=True, max_depth=90, max_features=3, min_samples_leaf=3, min_samples_split=8, n_estimators=1000; total time=   4.5s
[CV] END bootstrap=True, max_depth=90, max_features=3, min_samples_leaf=3, min_samples_split=10, n_estimators=1000; total time=   4.4s
[CV] END bootstrap=True, max_depth=90, max_features=3, min_samples_leaf=3, min_samples_split=12, n_estimators=300; total time=   1.3s
[CV] END bootstrap=True, max_depth=90, max_features=3, min_samples_leaf=3, min_samples_split=12, n_estimators=1000; total time=   4.6s
[CV] END bootstrap=True, max_depth=90, max_features=3, min_samples_leaf=4, min_samples_split=8, n_estimators=300; total time=   1.3s
[CV] END bootstrap=True, max_depth=90, max_features=3, min_samples_leaf=4, min_samples_split=8, n_estimators=1000; total time=   4.3s
[CV] END bootstrap=True, max_depth=90, max_features=3, min_sam

[CV] END bootstrap=True, max_depth=90, max_features=2, min_samples_leaf=5, min_samples_split=8, n_estimators=300; total time=   1.3s
[CV] END bootstrap=True, max_depth=90, max_features=2, min_samples_leaf=5, min_samples_split=8, n_estimators=1000; total time=   4.3s
[CV] END bootstrap=True, max_depth=90, max_features=2, min_samples_leaf=5, min_samples_split=10, n_estimators=300; total time=   1.3s
[CV] END bootstrap=True, max_depth=90, max_features=2, min_samples_leaf=5, min_samples_split=10, n_estimators=1000; total time=   4.2s
[CV] END bootstrap=True, max_depth=90, max_features=2, min_samples_leaf=5, min_samples_split=12, n_estimators=1000; total time=   4.2s
[CV] END bootstrap=True, max_depth=90, max_features=3, min_samples_leaf=3, min_samples_split=8, n_estimators=200; total time=   0.9s
[CV] END bootstrap=True, max_depth=90, max_features=3, min_samples_leaf=3, min_samples_split=8, n_estimators=1000; total time=   4.5s
[CV] END bootstrap=True, max_depth=90, max_features=3, min_sam

[CV] END bootstrap=True, max_depth=100, max_features=3, min_samples_leaf=5, min_samples_split=8, n_estimators=200; total time=   0.8s
[CV] END bootstrap=True, max_depth=100, max_features=3, min_samples_leaf=5, min_samples_split=8, n_estimators=200; total time=   0.9s
[CV] END bootstrap=True, max_depth=100, max_features=3, min_samples_leaf=5, min_samples_split=8, n_estimators=1000; total time=   4.3s
[CV] END bootstrap=True, max_depth=100, max_features=3, min_samples_leaf=5, min_samples_split=10, n_estimators=300; total time=   1.3s
[CV] END bootstrap=True, max_depth=100, max_features=3, min_samples_leaf=5, min_samples_split=10, n_estimators=1000; total time=   4.4s
[CV] END bootstrap=True, max_depth=100, max_features=3, min_samples_leaf=5, min_samples_split=12, n_estimators=300; total time=   1.3s
[CV] END bootstrap=True, max_depth=100, max_features=3, min_samples_leaf=5, min_samples_split=12, n_estimators=1000; total time=   4.4s
[CV] END bootstrap=True, max_depth=110, max_features=2,

RandomForestClassifier(max_depth=80, max_features=2, min_samples_leaf=5,
                       min_samples_split=12, n_estimators=300)


## Model with new parameters

In [44]:
# XGBoost

XGB = xgb.XGBClassifier(base_score=0.5, 
                        booster='gbtree',
                        callbacks=None,
                        colsample_bylevel=1,
                        colsample_bynode=1,
                        colsample_bytree=0.8,
                        early_stopping_rounds=None,
                        enable_categorical=False,
                        eval_metric=None,
                        feature_types=None,
                        gamma=0.4,
                        gpu_id=-1,
                        grow_policy='depthwise',
                        importance_type=None,
                        interaction_constraints='',
                        learning_rate=0.001,
                        max_bin=256,
                        max_cat_threshold=64,
                        max_cat_to_onehot=4,
                        max_delta_step=0, 
                        max_depth=3,
                        max_leaves=0,
                        min_child_weight=3, 
                        #missing=None,
                        monotone_constraints='()',
                        n_estimators=100, 
                        n_jobs=0,
                        num_parallel_tree=1,
                        nthread=None, 
                        objective='multi:softprob',
                        random_state=0, 
                        reg_alpha=1e-05, 
                        reg_lambda=1, 
                        scale_pos_weight=1,
                        seed=2, 
                        silent=True, 
                        subsample=0.8)
#XGB = xgb.XGBClassifier(user_label_encoder=False)
XGB.fit(X_train, y_train)
score_train_acc = XGB.score(X_train, y_train)
score_test_acc = XGB.score(X_test, y_test)
print(score_train_acc) #66.53
print(score_test_acc)  #56.59      
y_pred_XGB = XGB.predict(X_test)
print(classification_report(y_test, y_pred_XGB))
print(confusion_matrix(y_test, y_pred_XGB, labels=range(3)))

[22:29:55] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

0.6174334140435835
0.6214689265536724
              precision    recall  f1-score   support

           0       0.80      0.48      0.60       164
           1       0.54      0.90      0.67       145
           2       0.75      0.27      0.39        45

    accuracy                           0.62       354
   macro avg       0.70      0.55      0.55       354
weighted avg       0.69      0.62      0.60       354

[[ 78  84   2]
 [ 13 130   2]
 [  6  27  12]]


In [46]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
rc = RandomForestClassifier(max_depth=80,
                            max_features=2,
                            min_samples_leaf=5,
                            min_samples_split=12,
                            n_estimators=300)
rc.fit(X_train, y_train)
score_train_acc = rc.score(X_train, y_train)
score_test_acc = rc.score(X_test, y_test)
print(score_train_acc) #91.58
print(score_test_acc)  #51.31
y_pred_rc = rc.predict(X_test) 
print(classification_report(y_test, y_pred_rc))
print(confusion_matrix(y_test, y_pred_rc, labels=range(3)))

0.7130750605326877
0.6186440677966102
              precision    recall  f1-score   support

           0       0.70      0.58      0.64       164
           1       0.57      0.78      0.66       145
           2       0.52      0.24      0.33        45

    accuracy                           0.62       354
   macro avg       0.60      0.53      0.54       354
weighted avg       0.63      0.62      0.61       354

[[ 95  63   6]
 [ 28 113   4]
 [ 12  22  11]]


# Group Stage - all Kick-off

In [47]:
teams_ranking_df.head()

,rank,countryCode,countryName,totalPoints,previousPoints
0,1,BRA,Brazil,1841.30,1837.56
1,2,BEL,Belgium,1816.71,1821.92
2,3,ARG,Argentina,1773.88,1770.65
3,4,FRA,France,1759.78,1764.85
4,5,ENG,England,1728.47,1737.46


In [48]:
ReWrite_pred_df=pd.DataFrame(columns = ['Home Team Name',
                                        'Away Team Name',
                                        'home_team_rank',
                                        'home_team_points',
                                        'home_team_previous_points',
                                        'away_team_rank',
                                        'away_team_points',
                                        'away_team_previous_points',
                                        'Home Team Champion',
                                        'Away Team Champion',
                                        'Winner'])
ReWrite_pred_df.head()

,Home Team Name,Away Team Name,home_team_rank,home_team_points,home_team_previous_points,away_team_rank,away_team_points,away_team_previous_points,Home Team Champion,Away Team Champion,Winner


In [49]:

def prediction(team1, team2,df):
  ReWrite_pred_df=df
  id1 = team_name[team1]
  id2 = team_name[team2]
  championship1 = winners.get(team1) if winners.get(team1) != None else 0
  championship2 = winners.get(team2) if winners.get(team2) != None else 0
  
  team1_rank_1=teams_ranking_df[teams_ranking_df['countryName']==team1]['rank']
  team1_totalPoints_1=teams_ranking_df[teams_ranking_df['countryName']==team1]['totalPoints']
  team1_previousPoints_1=teams_ranking_df[teams_ranking_df['countryName']==team1]['previousPoints']   

  team2_rank=teams_ranking_df[teams_ranking_df['countryName']==team2]['rank']
  team2_totalPoints=teams_ranking_df[teams_ranking_df['countryName']==team2]['totalPoints']
  team2_previousPoints=teams_ranking_df[teams_ranking_df['countryName']==team2]['previousPoints']   
  
  t = np.array([id1, id2, team1_rank_1, team1_totalPoints_1, team1_previousPoints_1, team2_rank, team2_totalPoints, team2_previousPoints, championship1, championship2]).astype('float64')
  t = np.reshape(t, (1,-1))
  y_rc = XGB.predict_proba(t)[0]
    
  home_team_rank=teams_ranking_df[teams_ranking_df['countryName']==team1]['rank']
  home_team_points=teams_ranking_df[teams_ranking_df['countryName']==team1]['totalPoints']
  home_team_previous_points=teams_ranking_df[teams_ranking_df['countryName']==team1]['previousPoints']
  away_team_rank=teams_ranking_df[teams_ranking_df['countryName']==team2]['rank']
  away_team_points=teams_ranking_df[teams_ranking_df['countryName']==team2]['totalPoints']
  away_team_previous_points=teams_ranking_df[teams_ranking_df['countryName']==team2]['previousPoints']
  Home_Team_Champion=(winners.get(team1) if winners.get(team1) != None else 0)
  Away_Team_Champion=(winners.get(team2) if winners.get(team2) != None else 0)
  conditions = [y_rc[0]> y_rc[1] and y_rc[0]> y_rc[2],
               y_rc[1]> y_rc[2],
               y_rc[2]>y_rc[1]]
  values =[0,1,2]

  Winner=np.select(conditions,values)

  ReWrite_pred_df=ReWrite_pred_df.append({'Home Team Name':id1,
                               'Away Team Name':id2,
                               'home_team_rank':home_team_rank.values[0],
                               'home_team_points':home_team_points.values[0],
                               'home_team_previous_points':home_team_previous_points.values[0],
                               'away_team_rank':away_team_rank.values[0],
                               'away_team_points':away_team_points.values[0],
                               'away_team_previous_points':away_team_previous_points.values[0],
                               'Home Team Champion':Home_Team_Champion,
                               'Away Team Champion':Away_Team_Champion,
                               'Winner':Winner},
                               ignore_index = True)

  text = ('Chance for '+team1+' to win against '+team2+' is {}\nChance for '+team2+' to win against '+team1+' is {}\nChance for '+team1+' and '+team2+' draw is {}').format(y_rc[1]*100,y_rc[2]*100,y_rc[0]*100)
  return y_rc, text,ReWrite_pred_df

In [50]:
probs, text, ReWrite_pred_df = prediction('Senegal', 'Portugal',ReWrite_pred_df)
print(text)
ReWrite_pred_df.head()

Chance for Senegal to win against Portugal is 32.079872488975525
Chance for Portugal to win against Senegal is 32.92520046234131
Chance for Senegal and Portugal draw is 34.994930028915405


/tmp/ipykernel_30404/3334352355.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  t = np.array([id1, id2, team1_rank_1, team1_totalPoints_1, team1_previousPoints_1, team2_rank, team2_totalPoints, team2_previousPoints, championship1, championship2]).astype('float64')
/tmp/ipykernel_30404/3334352355.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ReWrite_pred_df=ReWrite_pred_df.append({'Home Team Name':id1,


,Home Team Name,Away Team Name,home_team_rank,home_team_points,home_team_previous_points,away_team_rank,away_team_points,away_team_previous_points,Home Team Champion,Away Team Champion,Winner
0,65,37,18,1584.38,1584.59,9,1676.56,1678.65,0,0,0


In [51]:
Team_id_updated = Team_id.append(ReWrite_pred_df,ignore_index = True)
Team_id_updated.tail()

/tmp/ipykernel_30404/2971977594.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Team_id_updated = Team_id.append(ReWrite_pred_df,ignore_index = True)


,Home Team Name,Away Team Name,home_team_rank,home_team_points,home_team_previous_points,away_team_rank,away_team_points,away_team_previous_points,Home Team Champion,Away Team Champion,Winner
1176,65,15,18,1584.38,1584.59,39.0,1486.44,1482.63,0,0,0
1177,65,15,18,1584.38,1584.59,39.0,1486.44,1482.63,0,0,1
1178,42,7,38,1488.72,1483.73,23.0,1561.01,1562.24,0,0,0
1179,9,74,29,1509.61,1514.86,61.0,1393.00,1393.47,0,0,0
1180,65,37,18,1584.38,1584.59,9,1676.56,1678.65,0,0,0


## Retraining the model with the new predicted data

In [52]:
# Preparing Training and Testing Data:
X = Team_id_updated.iloc[:, 0:10].values
y = Team_id_updated.iloc[:, -1].values

X = np.array(X, dtype = 'f')
y = np.array(y, dtype = 'f')

from sklearn.preprocessing import LabelEncoder
LabelEncoder_res=LabelEncoder()
y=LabelEncoder_res.fit_transform(y)

X, y = shuffle(X,y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

##  Rescaling the model with the trained data

In [53]:
# Scalling of Data:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

## Rebuilding an ANN

In [54]:
# Building ANN
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

## Re Seeding

In [55]:
#seed
np.random.seed(47)

## Re-Random Forest

In [57]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
rc = RandomForestClassifier(max_depth=80,
                            max_features=2,
                            min_samples_leaf=5,
                            min_samples_split=12,
                            n_estimators=300)
rc.fit(X_train, y_train)
score_train_acc = rc.score(X_train, y_train)
score_test_acc = rc.score(X_test, y_test)
print(score_train_acc) #91.58
print(score_test_acc)  #51.31
y_pred_rc = rc.predict(X_test) 
print(classification_report(y_test, y_pred_rc))
print(confusion_matrix(y_test, y_pred_rc, labels=range(3)))

0.7094430992736077
0.5971830985915493
              precision    recall  f1-score   support

           0       0.63      0.59      0.61       160
           1       0.59      0.71      0.64       151
           2       0.44      0.25      0.32        44

    accuracy                           0.60       355
   macro avg       0.55      0.52      0.52       355
weighted avg       0.59      0.60      0.59       355

[[ 94  56  10]
 [ 40 107   4]
 [ 15  18  11]]


## Re XGBOOST

In [58]:
# XGBoost

XGB = xgb.XGBClassifier(base_score=0.5, 
                        booster='gbtree',
                        callbacks=None,
                        colsample_bylevel=1,
                        colsample_bynode=1,
                        colsample_bytree=0.8,
                        early_stopping_rounds=None,
                        enable_categorical=False,
                        eval_metric=None,
                        feature_types=None,
                        gamma=0.4,
                        gpu_id=-1,
                        grow_policy='depthwise',
                        importance_type=None,
                        interaction_constraints='',
                        learning_rate=0.001,
                        max_bin=256,
                        max_cat_threshold=64,
                        max_cat_to_onehot=4,
                        max_delta_step=0, 
                        max_depth=3,
                        max_leaves=0,
                        min_child_weight=3, 
                        #missing=None,
                        monotone_constraints='()',
                        n_estimators=100, 
                        n_jobs=0,
                        num_parallel_tree=1,
                        nthread=None, 
                        objective='multi:softprob',
                        random_state=0, 
                        reg_alpha=1e-05, 
                        reg_lambda=1, 
                        scale_pos_weight=1,
                        seed=2, 
                        silent=True, 
                        subsample=0.8)
#XGB = xgb.XGBClassifier(user_label_encoder=False)
XGB.fit(X_train, y_train)
score_train_acc = XGB.score(X_train, y_train)
score_test_acc = XGB.score(X_test, y_test)
print(score_train_acc) #66.53
print(score_test_acc)  #56.59      
y_pred_XGB = XGB.predict(X_test)
print(classification_report(y_test, y_pred_XGB))
print(confusion_matrix(y_test, y_pred_XGB, labels=range(3)))

[22:39:30] WARNING: ../src/learner.cc:767: 
Parameters: { "scale_pos_weight", "silent" } are not used.

0.6174334140435835
0.6253521126760564
              precision    recall  f1-score   support

           0       0.80      0.46      0.58       160
           1       0.57      0.91      0.70       151
           2       0.55      0.25      0.34        44

    accuracy                           0.63       355
   macro avg       0.64      0.54      0.54       355
weighted avg       0.67      0.63      0.60       355

[[ 73  81   6]
 [ 10 138   3]
 [  8  25  11]]


## Repredict

In [59]:
probs, text, ReWrite_pred_df = prediction('USA', 'Brazil',ReWrite_pred_df)
print(text)
ReWrite_pred_df.head()

Chance for USA to win against Brazil is 32.02051222324371
Chance for Brazil to win against USA is 33.6462676525116
Chance for USA and Brazil draw is 34.33322012424469


/tmp/ipykernel_30404/3334352355.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  t = np.array([id1, id2, team1_rank_1, team1_totalPoints_1, team1_previousPoints_1, team2_rank, team2_totalPoints, team2_previousPoints, championship1, championship2]).astype('float64')
/tmp/ipykernel_30404/3334352355.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ReWrite_pred_df=ReWrite_pred_df.append({'Home Team Name':id1,


,Home Team Name,Away Team Name,home_team_rank,home_team_points,home_team_previous_points,away_team_rank,away_team_points,away_team_previous_points,Home Team Champion,Away Team Champion,Winner
0,65,37,18,1584.38,1584.59,9,1676.56,1678.65,0,0,0
1,2,5,16,1627.48,1635.01,1,1841.30,1837.56,0,5,0
